# Utils

In [94]:
from collections import defaultdict
from math import sin, cos, e, log
from timeit import default_timer as timer
from typing import Callable, Tuple, Literal, Optional

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.sparse import lil_matrix, vstack, coo_matrix
from scipy.sparse.linalg import spsolve
from tqdm import tqdm

import plotly.express as px

from functools import partialmethod

tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

In [95]:
Function = Callable[[float], float]
Grid = Tuple[np.ndarray, float]
GeneralCondition = Tuple[float, float, float]

COLORS = px.colors.qualitative.Dark24

In [96]:
def solve(
        matrix: lil_matrix,
        right_side: np.ndarray,
        matrix_type: Literal['tridiagonal', 'other'] = 'other',
) -> np.ndarray:
    if matrix_type == 'other':
        return spsolve(matrix, right_side)

    alphas = []
    betas = []

    # Прямой ход: находим коэффициенты
    for i in tqdm(range(matrix.shape[0]), desc='Прямой ход'):
        y = matrix[i, i] if i == 0 else matrix[i, i] + matrix[i, i - 1] * alphas[i - 1]
        alphas.append(0 if i == (matrix.shape[0] - 1) else -1 * matrix[i, i + 1] / y)
        betas.append(right_side[i] / y if i == 0 else (right_side[i] - matrix[i, i - 1] * betas[i - 1]) / y)

    # Обратный ход: находим x
    reversed_solution = []
    for i in tqdm(reversed(range(matrix.shape[0])), desc='Обратный ход'):
        reversed_solution.append(
            betas[i]
            if i == (matrix.shape[0] - 1) else
            alphas[i] * reversed_solution[matrix.shape[0] - i - 2] + betas[i]
        )
    return np.array(reversed_solution[::-1])

In [97]:
def solve_equation_using_finite_difference_method(
        q: Function,
        r: Function,
        f: Function,
        grid: Grid,
        left_general_condition: GeneralCondition,
        right_general_condition: GeneralCondition,
) -> np.ndarray:
    left_coefficient_1, left_coefficient_2, left_value = left_general_condition
    right_coefficient_1, right_coefficient_2, right_value = right_general_condition

    matrix_type = 'tridiagonal' if left_coefficient_2 == 0 and right_coefficient_2 == 0 else 'other'

    points, step = grid

    first_row = [
        left_coefficient_1 + 3 / (2 * step) * left_coefficient_2,
        -2 * left_coefficient_2 / step,
        left_coefficient_2 / (2 * step),
    ]
    first_row.extend([0] * (len(points) - 3))

    matrix = coo_matrix([first_row])
    right_side = [left_value]

    for index, point in tqdm(enumerate(points[1:-1]), desc='Конвертация в СЛАУ'):
        row = [0] * index

        row.append(1 / (step ** 2) - q(point) / (2 * step))
        row.append(-2 / (step ** 2) - r(point))
        row.append(1 / (step ** 2) + q(point) / (2 * step))

        row.extend([0] * (len(points) - index - 3))

        matrix = vstack([matrix, row])
        right_side.append(f(point))

    last_row = [float(0)] * (len(points) - 3)
    last_row.extend([
        right_coefficient_2 / (2 * step),
        -2 * right_coefficient_2 / step,
        right_coefficient_1 + 3 * right_coefficient_2 / (2 * step),
    ])

    matrix = vstack([matrix, last_row], format='lil')
    right_side.append(right_value)

    return solve(matrix, np.array(right_side), matrix_type)

In [98]:
def _get_richardson_error(curr_solution: np.ndarray, next_solution: np.ndarray) -> float:
    error_by_even_points = abs(next_solution[::2] - curr_solution) / 3

    max_error = -1
    for left_error, right_error in zip(error_by_even_points, error_by_even_points[1:]):
        max_error = max(max_error, left_error)
        max_error = max(max_error, (left_error + right_error) / 2)
        max_error = max(max_error, right_error)

    return max_error


def _get_true_error(true_solution: np.ndarray, actual_solution: np.ndarray) -> float:
    return abs(true_solution - actual_solution).max()


def banchmark(
        q: Function,
        r: Function,
        f: Function,
        a: float,
        b: float,
        left_general_condition: GeneralCondition,
        right_general_condition: GeneralCondition,
        checkpoint_name: str,
        true_function: Optional[Callable[[float], float]] = None,
        eps: Optional[float] = None,
        limit: int = 20,
) -> None:
    data = pd.read_csv(checkpoint_name)

    if true_function is not None:
        true_function = np.vectorize(true_function)

    number_of_points = 3

    curr_grid = np.linspace(a, b, number_of_points, retstep=True)

    if data[data.number_of_points == number_of_points].empty:
        curr_solution = solve_equation_using_finite_difference_method(
            q,
            r,
            f,
            curr_grid,
            left_general_condition,
            right_general_condition
        )
    else:
        curr_solution = np.array(eval(data.at[data.number_of_points == number_of_points, 'solution'].values[0]))

    result = defaultdict(list)

    for step in range(2, limit + 1):
        print(f'Current step: {step}', end=' ')

        number_of_points = number_of_points * 2 - 1

        next_grid = np.linspace(a, b, number_of_points, retstep=True)

        if data[data.number_of_points == number_of_points].empty:
            start = timer()
            next_solution = solve_equation_using_finite_difference_method(
                q,
                r,
                f,
                next_grid,
                left_general_condition,
                right_general_condition
            )
            end = timer()
            time = end - start
            print('(new)')
        else:
            next_solution = np.array(eval(data.loc[data.number_of_points == number_of_points, 'solution'].values[0]))
            time = data.loc[data.number_of_points == number_of_points, 'time'].values[0]
            print('(saved)')

        result['step'].append(step)
        result['number_of_points'].append(number_of_points)

        error = _get_richardson_error(curr_solution, next_solution)
        result['richardson_error'].append(error)

        if true_function is not None:
            result['true_error'].append(_get_true_error(true_function(next_grid[0]), next_solution))

        result['time'].append(time)
        result['solution'].append(next_solution.tolist())

        pd.DataFrame.from_dict(result).to_csv(checkpoint_name, index=False)

        curr_grid = next_grid
        curr_solution = next_solution

        if eps is not None and error <= eps:
            break

In [99]:
def compare_step_and_error(data: pd.DataFrame) -> None:
    fig = go.Figure()

    fig.add_scatter(x=data.step, y=data.richardson_error, name='По Ричардсону')

    if 'true_error' in data.columns.tolist():
        fig.add_scatter(x=data.step, y=data.true_error, name='Реальная')

    fig.update_xaxes(title='Номер шага')
    fig.update_yaxes(title='Ошибка', tickformat='.2e', type='log')

    fig.update_layout(title='Зависимость ошибки от номера шага', legend_title='Ошибка')

    fig.show()

In [100]:
def compare_step_and_number_of_points(data: pd.DataFrame) -> None:
    fig = go.Figure()

    fig.add_scatter(x=data.step, y=data.number_of_points)

    fig.update_xaxes(title='Номер шага')
    fig.update_yaxes(title='Количество точек')

    fig.update_layout(title='Зависимость количества точек в сетке от номера шага')

    fig.show()

In [101]:
def show_solution_approximations(
        data: pd.DataFrame,
        a: int,
        b: int,
        start_with_step=2,
        end_with_step=20,
        true_function: Optional[Callable[[float], float]] = None,
        number_of_points: int = 131073,
        x_bias: float = 1,
) -> None:
    fig = go.Figure()

    data = data[(data.step >= start_with_step) & (data.step <= end_with_step)]

    data.apply(
        func=lambda row: fig.add_scatter(
            x=np.linspace(a, b, row.number_of_points),
            y=eval(row.solution),
            name=row.step,
            line_color=COLORS[row.name],
            mode='lines',
        ),
        axis=1,
    )

    if true_function is not None:
        x, step = np.linspace(a, b, number_of_points, retstep=True)
        fig.add_scatter(x=x, y=list(map(true_function, x)), name='True', line_color='rgba(0, 0, 0, 0.25)', showlegend=False)

        x = np.linspace(a - x_bias, a, int(1000 * x_bias))
        fig.add_scatter(x=x, y=list(map(true_function, x)), name='True', line_color='rgba(0, 0, 0, 0.25)', showlegend=False)

        x = np.linspace(b, b + x_bias, int(1000 * x_bias))
        fig.add_scatter(x=x, y=list(map(true_function, x)), name='True', line_color='rgba(0, 0, 0, 0.25)', showlegend=False)

    fig.update_layout(title='Приближение решения', legend_title='Шаг')

    fig.show(config={'scrollZoom': True})

In [102]:
def compare_step_and_time(data: pd.DataFrame) -> None:
    fig = go.Figure()

    fig.add_scatter(x=data.step, y=data.time / 60)

    fig.update_xaxes(title='Номер шага')
    fig.update_yaxes(title='Время (мин.)')

    fig.update_layout(title='Зависимость времени нахождения решения от номера шага')

    fig.show()

# Simple conditions

$$u = x e^{sin(x)}$$
$$u(-2) = -2 e^{sin(-2)}$$
$$u(6) = 6 e^{sin(6)}$$

In [103]:
u = lambda x: x * e ** sin(x)
du = lambda x: e ** sin(x) * (x * cos(x) + 1)
ddu = lambda x: e ** sin(x) * (-x * sin(x) + x * cos(x) ** 2 + 2 * cos(x))

q = lambda x: -x
r = lambda x: 1
f = lambda x: ddu(x) + q(x) * du(x) - r(x) * u(x)

a = -2
b = 6

alpha_1 = 1
alpha_2 = 0
alpha = -2 * e ** sin(-2)

left_condition = (alpha_1, alpha_2, alpha)

beta_1 = 1
beta_2 = 0
beta = 6 * e ** sin(6)

right_condition = (beta_1, beta_2, beta)

In [104]:
# banchmark(
#     q,
#     r,
#     f,
#     a,
#     b,
#     left_condition,
#     right_condition,
#     true_function=u,
#     checkpoint_name='simple.csv',
# )

In [105]:
data = pd.read_csv('simple.csv')

compare_step_and_number_of_points(data)
compare_step_and_time(data)
compare_step_and_error(data)

Unsupported

In [106]:
data = pd.read_csv('simple.csv')

u = lambda x: x * e ** sin(x)

show_solution_approximations(data, -2, 6, true_function=u, end_with_step=5)

Unsupported

In [107]:
data = pd.read_csv('simple.csv')

u = lambda x: x * e ** sin(x)

show_solution_approximations(data, -2, 6, true_function=u, start_with_step=17)

Unsupported

# General conditions

$$u(x) = x^5 - 5x^4 + 5x^3 + 5x^2 - 6x$$
$$2u(-1) - 3u'(-1) = -24$$
$$3u(3) + \frac{3}{8}u'(3) = 3$$

In [108]:
u = lambda x: x ** 5 - 5 * x ** 4 + 5 * x ** 3 + 5 * x ** 2 - 6 * x
du = lambda x: 5 * x ** 4 - 20 * x ** 3 + 15 * x ** 2 + 10 * x - 6
ddu = lambda x: 10 * (2 * x ** 3 - 6 * x ** 2 + 3 * x + 1)

q = lambda x: 5
r = lambda x: -x
f = lambda x: ddu(x) + q(x) * du(x) - r(x) * u(x)

a = -1
b = 3

alpha_1 = 2
alpha_2 = 1
alpha = -24

left_condition = (alpha_1, alpha_2, alpha)

beta_1 = 3
beta_2 = 1 / 8
beta = 3

right_condition = (beta_1, beta_2, beta)

In [109]:
# banchmark(
#     q,
#     r,
#     f,
#     a,
#     b,
#     left_condition,
#     right_condition,
#     true_function=u,
#     checkpoint_name='general.csv',
# )

In [110]:
data = pd.read_csv('general.csv')

compare_step_and_number_of_points(data)
compare_step_and_time(data)
compare_step_and_error(data)

Unsupported

In [111]:
data = pd.read_csv('general.csv')

u = lambda x: x ** 5 - 5 * x ** 4 + 5 * x ** 3 + 5 * x ** 2 - 6 * x

show_solution_approximations(data, -1, 3, true_function=u, end_with_step=7, x_bias=0.25)

Unsupported

In [112]:
data = pd.read_csv('general.csv')

u = lambda x: x ** 5 - 5 * x ** 4 + 5 * x ** 3 + 5 * x ** 2 - 6 * x

show_solution_approximations(data, -1, 3, true_function=u, start_with_step=14, end_with_step=16)

Unsupported

# Pakulina (20)

$$-\frac{7 + x}{8 - 3x}u'' + \left(1 - \frac{x}{3} \right)u' + \frac{1}{3}\text{ln}(3 + x)u = \frac{1 + x}{2}$$
$$u(-1)-3u'(-1) = 0$$
$$u'(1) = 0$$

In [113]:
k = lambda x: -1 * (7 + x) / (8 - 3 * x)
q = lambda x: (1 - x / 3) / k(x)
r = lambda x: -1 / 3 * log(3 + x) / k(x)
f = lambda x: (1 + x) / (2 * k(x))

a = -1
b = 1

alpha_1 = 1
alpha_2 = 3
alpha = 0

left_condition = (alpha_1, alpha_2, alpha)

beta_1 = 0
beta_2 = 1
beta = 0

right_condition = (beta_1, beta_2, beta)

In [114]:
# banchmark(
#     q,
#     r,
#     f,
#     a,
#     b,
#     left_condition,
#     right_condition,
#     checkpoint_name='pakulina.csv',
# )

In [115]:
data = pd.read_csv('pakulina.csv')

compare_step_and_number_of_points(data)
compare_step_and_time(data)
compare_step_and_error(data)

Unsupported

In [116]:
data = pd.read_csv('pakulina.csv')

show_solution_approximations(data, -1, 1, end_with_step=6)

Unsupported

In [117]:
data = pd.read_csv('pakulina.csv')
show_solution_approximations(data, -1, 1, start_with_step=15, end_with_step=17)

Unsupported